In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!python --version

Python 3.10.12


In [3]:
%cd /content/drive/MyDrive/enz-eff-project
!pip install -r requirements.txt

/content/drive/.shortcut-targets-by-id/1iS6gSWfUE3cZnmrNWbbV9_W_zQH3vaiu/enz-eff-project
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 16.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.2/101.2 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 72.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.9/191.9 kB 19.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9

In [4]:
%cd improved_code/preprocessing

/content/drive/.shortcut-targets-by-id/1iS6gSWfUE3cZnmrNWbbV9_W_zQH3vaiu/enz-eff-project/improved_code/preprocessing


In [ ]:
!pip install pandas==1.5.3

In [5]:
import pandas as pd
import numpy as np
from os.path import join
import os
from rdkit import Chem
from rdkit.Chem import AllChem
from drfp import DrfpEncoder
from rdkit.Chem import Crippen
from rdkit.Chem import Descriptors
from bioservices import *
from data_preprocessing import *
import warnings
warnings.filterwarnings('ignore')

CURRENT_DIR = os.getcwd()

Creating directory /root/.config/bioservices 


## 1. Merging the kcat datasets from BRENDA, Sabio-RK, and UniProt:

### (a) Loading BRENDA data

In [ ]:
df_Brenda = pd.read_pickle(join("..", "..", "data", "kcat_data", "BRENDA_kcat.pkl"))
#adding reaction information:
df_Brenda.rename(columns = {"correct reaction ID" : "BRENDA reaction ID"})


df_Brenda["Uniprot ID"] = np.nan
for ind in df_Brenda.index:
    try:
        df_Brenda["Uniprot ID"][ind] = df_Brenda["UNIPROT_list"][ind][0]
    except IndexError:
        pass

df_Brenda = df_Brenda.loc[~pd.isnull(df_Brenda["Uniprot ID"])]


df_Brenda.drop(columns = ["index", "ID", "comment", "kcat", "kcat_new", "enzyme",
                         "new", "LITERATURE", "UNIPROT_list", "new enzyme"],
               inplace = True)

df_Brenda.rename(columns = {"correct kcat" : "kcat", "correct reaction ID" : "BRENDA reaction ID",
                           "substrate_ID_list" : "substrate_IDs",
                           "product_ID_list" : "product_IDs"}, inplace = True)


print("Number of data points: %s" % len(df_Brenda))
print("Number of UniProt IDs: %s" % len(set(df_Brenda["Uniprot ID"])))
print("Number of checked data points: %s" % len(df_Brenda.loc[df_Brenda["checked"]]))
print("Number of unchecked data points: %s" % len(df_Brenda.loc[~df_Brenda["checked"]]))


df_Brenda["from BRENDA"] = 1
df_Brenda["from Uniprot"] = 0
df_Brenda["from Sabio"] = 0
df_Brenda.head()

Number of data points: 8267
Number of UniProt IDs: 3149
Number of checked data points: 3611
Number of unchecked data points: 4656


,EC,ORGANISM,PMID,BRENDA reaction ID,kcat,checked,#UIDs,substrate_IDs,product_IDs,Uniprot ID,from BRENDA,from Uniprot,from Sabio
44,1.1.1.363,Leuconostoc mesenteroides,1304341.0,1485,1125.0,True,1,[InChI=1S/C21H27N7O14P2/c22-17-12-19(25-7-24-1...,"[InChI=1S/p+1, InChI=1S/C21H29N7O14P2/c22-17-1...",P11411,1,0,0
45,3.6.1.1,Thermoplasma acidophilum,1327774.0,26801,2200.0,True,1,"[InChI=1S/H2O/h1H2, InChI=1S/H4O7P2/c1-8(2,3)7...","[InChI=1S/H3O4P/c1-5(2,3)4/h(H3,1,2,3,4)/p-3]",P37981,1,0,0
64,2.5.1.7,Escherichia coli,1512209.0,12872,4.75,True,1,"[InChI=1S/C3H5O6P/c1-2(3(4)5)9-10(6,7)8/h1H2,(...","[InChI=1S/H3O4P/c1-5(2,3)4/h(H3,1,2,3,4)/p-3, ...",P0A749,1,0,0
65,1.12.98.2,Methanothermobacter marburgensis,1521540.0,3791,1462.0,True,1,"[InChI=1S/p+1, InChI=1S/C31H45N6O16P/c1-13-22-...","[InChI=1S/H2/h1H, InChI=1S/C31H43N6O16P/c1-13-...",P32440,1,0,0
84,4.1.3.40,Escherichia coli,1644758.0,28554,0.82,True,1,[InChI=1S/C10H10O6/c1-5(9(12)13)16-8-4-6(10(14...,"[InChI=1S/C3H4O3/c1-2(4)3(5)6/h1H3,(H,5,6)/p-1...",P26602,1,0,0


### (b) Loading Sabio data

In [ ]:
df_Sabio = pd.read_pickle(join("..", "..", "data", "kcat_data", "Sabio_kcat.pkl"))
df_Sabio.drop(columns = ["unit", "complete", "KEGG ID"], inplace = True)
df_Sabio.rename(columns = {"products_IDs": "product_IDs"}, inplace = True)

print("Number of data points: %s" % len(df_Sabio))
print("Number of UniProt IDs: %s" % len(set(df_Sabio["Uniprot ID"])))

df_Sabio["checked"] = False
df_Sabio["#UIDs"] = 1
df_Sabio["complete"] = True

df_Sabio["from BRENDA"] = 0
df_Sabio["from Uniprot"] = 0
df_Sabio["from Sabio"] = 1
df_Sabio.head()

### (c) Loading UniProt data:

In [ ]:
df_Uniprot = pd.read_pickle(join("..", "..", "data", "kcat_data", "Uniprot_kcat.pkl"))

df_Uniprot.drop(columns = ["unit", "reaction ID"], inplace = True)
df_Uniprot.rename(columns = {"substrate CHEBI IDs" : "Substrates", "product CHEBI IDs" : "Products",
                            "substrate InChIs" : "substrate_IDs", "product InChIs" : "product_IDs",
                            "kcat [1/sec]" : "kcat"}, inplace = True)

print("Number of data points: %s" % len(df_Uniprot))
print("Number of UniProt IDs: %s" % len(set(df_Uniprot["Uniprot ID"])))

df_Uniprot["checked"] = False
df_Uniprot["#UIDs"] = 1

df_Uniprot["from BRENDA"] = 0
df_Uniprot["from Uniprot"] = 1
df_Uniprot["from Sabio"] = 0
df_Uniprot.head()

### (d) Merging all three datasets

In [ ]:
df_kcat = pd.concat([pd.concat([df_Sabio, df_Brenda], ignore_index = True), df_Uniprot], ignore_index = True)
df_kcat =df_kcat.loc[~pd.isnull(df_kcat["kcat"])]

print("Number of data points: %s" % len(df_kcat))
print("Number of UniProt IDs: %s" % len(set(df_kcat["Uniprot ID"])))
df_kcat.to_pickle(join("..", "..", "data", "kcat_data", "kcat_data_merged.pkl"))
df_kcat.head(2)

### (e) Removing duplicated entries:

In [ ]:
df_kcat = pd.read_pickle(join("..", "..", "data", "kcat_data", "kcat_data_merged.pkl"))

#### Searching for identitcal pairs of UniProt IDs and kcat values:

In [ ]:
droplist = []

for ind in df_kcat.index:
    UID, kcat = df_kcat["Uniprot ID"][ind], df_kcat["kcat"][ind]
    help_df = df_kcat.loc[df_kcat["Uniprot ID"] == UID].loc[df_kcat["kcat"] == kcat]

    df_kcat["from BRENDA"][ind], df_kcat["from Uniprot"][ind], df_kcat["from Sabio"][ind] = max(help_df["from BRENDA"]), max(help_df["from Uniprot"]), max(help_df["from Sabio"])
    df_kcat["checked"][ind] = any(help_df["checked"])

    if len(help_df) > 1:
        droplist = droplist + list(help_df.index)[1:]

In [ ]:
df_kcat.drop(list(set(droplist)), inplace = True)
print("Dropping %s data points, because they are duplicated." % len(set(droplist)))

## 2. Downloading amino acid sequences for all data points:

### (a) Downloading sequences via UniProt IDs:

Creating a txt file with all Uniprot IDs

In [ ]:
IDs = list(set(df_kcat["Uniprot ID"]))
f = open(join("..", "..", "data", "enzyme_data", "UNIPROT_IDs.txt"), "w")
for ID in list(set(IDs)):
    f.write(str(ID) + "\n")
f.close()

Mapping Uniprot IDs to sequences via the UniProt mapping service and saving the results in the file "UNIPROT_results.tab"

In [ ]:
UNIPROT_df = pd.read_csv(join("..", "..", "data", "enzyme_data",  "UNIPROT_results.tab"), sep = "\t")
UNIPROT_df.drop(columns = ["Entry"], inplace = True)
display(UNIPROT_df.head())

df_kcat = df_kcat.merge(UNIPROT_df, how = "left", on = "Uniprot ID")
df_kcat = df_kcat.loc[~pd.isnull(df_kcat["Uniprot ID"])]

In [ ]:
print("Number of different amino acid sequences in the dataset: %s" % len(set(df_kcat["Sequence"])))

## 3. Mapping all substrates and products to InChI strings:
#### Most of the metabolites in our dataset have InChI strings as identifiers and some of them have KEGG Compound IDs. We are trying to map the KEGG Compound IDs to InChI strings as well:

#### (a) Getting an InChI string for all metabolites

In [ ]:
kegg_con = KEGG()
chebi_con = ChEBI()

met_IDs = []

for ind in df_kcat.index:
    sub_IDs, pro_IDs = df_kcat["substrate_IDs"][ind], df_kcat["product_IDs"][ind]
    if sub_IDs != "" and pro_IDs != "" :
        try:
            met_IDs = met_IDs +sub_IDs + pro_IDs
        except TypeError:
            pass

df_metabolites = pd.DataFrame(data = {"metabolite ID": list(set(met_IDs))})
df_metabolites = df_metabolites.loc[df_metabolites["metabolite ID"] != ""]
df_metabolites["InChI"] = np.nan

for ind in df_metabolites.index:
    met = df_metabolites["metabolite ID"][ind]
    if met[0:5] == "InChI":
        df_metabolites["InChI"][ind] = met
    else:
        try:
            kegg_entry = kegg_con.parse(kegg_con.get(met))
            chebi_entry = chebi_con.getCompleteEntity('CHEBI:' + kegg_entry['DBLINKS']['ChEBI'])
            df_metabolites["InChI"][ind] = chebi_entry.inchi
        except:
            pass

df_metabolites.head()

In [ ]:
from os.path import join


for ind in df_metabolites.index:
    if pd.isnull(df_metabolites["InChI"][ind]):
        try:
            mol = Chem.MolFromMolFile(join("..", "..", "data", "metabolite_data", "mol-files", df_metabolites["metabolite ID"][ind] + '.mol'))
            df_metabolites["InChI"][ind] = Chem.MolToInchi(mol)
        except:
            pass

df_metabolites = df_metabolites.loc[~pd.isnull(df_metabolites["InChI"])]

#### (b) Mapping the InChI strings for all substrates and all products to the kcat values:

In [ ]:
df_kcat["substrate_InChI_set"] = ""
df_kcat["product_InChI_set"] = ""

for ind in df_kcat.index:
    sub_IDs, pro_IDs = df_kcat["substrate_IDs"][ind], df_kcat["product_IDs"][ind]

    try:
        sub_inchis = []
        pro_inchis = []
        for sub in sub_IDs:
            inchi = list(df_metabolites["InChI"].loc[df_metabolites["metabolite ID"] == sub])[0]
            sub_inchis.append(inchi)
        for pro in pro_IDs:
            inchi = list(df_metabolites["InChI"].loc[df_metabolites["metabolite ID"] == pro])[0]
            pro_inchis.append(inchi)

        df_kcat["substrate_InChI_set"][ind] = set(sub_inchis)
        df_kcat["product_InChI_set"][ind] = set(pro_inchis)
    except:
        pass

## 4. Assigning  IDs to every unique sequence and to every unique reaction in the dataset:

#### (a) Creating DataFrames for all sequences and for all reactions:

In [ ]:
df_sequences = pd.DataFrame(data = {"Sequence" : list(set(df_kcat["Sequence"]))})
df_sequences = df_sequences.loc[~pd.isnull(df_sequences["Sequence"])]
df_sequences.reset_index(inplace = True, drop = True)
df_sequences["Sequence ID"] = ["Sequence_" + str(ind) for ind in df_sequences.index]

df_sequences

In [ ]:
df_reactions = pd.DataFrame({"substrates": df_kcat["substrate_InChI_set"],
                            "products" : df_kcat["product_InChI_set"]})
df_reactions = df_reactions.loc[df_reactions["substrates"] != set([])]
df_reactions = df_reactions.loc[df_reactions["products"] != set([])]


droplist = []
for ind in df_reactions.index:
    sub_IDs, pro_IDs = df_reactions["substrates"][ind], df_reactions["products"][ind]
    help_df = df_reactions.loc[df_reactions["substrates"] == sub_IDs].loc[df_reactions["products"] == pro_IDs]
    if len(help_df):
        for ind in list(help_df.index)[1:]:
            droplist.append(ind)

df_reactions.drop(list(set(droplist)), inplace = True)
df_reactions.reset_index(inplace = True, drop =True)

df_reactions["Reaction ID"] = ["Reaction_" + str(ind) for ind in df_reactions.index]
df_reactions

#### (b) Calcuating the sum of the molecular weights of all substrates and of all products:

In [ ]:
df_reactions["MW_frac"] = np.nan

for ind in df_reactions.index:
    substrates = list(df_reactions["substrates"][ind])
    products = list(df_reactions["products"][ind])

    mw_subs = mw_mets(metabolites = substrates)
    mw_pros = mw_mets(metabolites = products)
    if mw_pros != 0:
        df_reactions["MW_frac"][ind] = mw_subs/mw_pros
    else:
        df_reactions["MW_frac"][ind] = np.inf

df_reactions

#### (c) Mapping Sequence and Reaction IDs to kcat_df:

In [ ]:
df_kcat = df_kcat.merge(df_sequences, on = "Sequence", how = "left")

In [ ]:
df_reactions.rename(columns = {"substrates" : "substrate_InChI_set",
                              "products" : "product_InChI_set"}, inplace = True)

df_kcat["Reaction ID"] = np.nan
df_kcat["MW_frac"] = np.nan
for ind in df_kcat.index:
    sub_set, pro_set = df_kcat["substrate_InChI_set"][ind], df_kcat["product_InChI_set"][ind]

    help_df = df_reactions.loc[df_reactions["substrate_InChI_set"] == sub_set].loc[df_reactions["product_InChI_set"] == pro_set]
    if len(help_df) == 1:
        df_kcat["Reaction ID"][ind] = list(help_df["Reaction ID"])[0]
        df_kcat["MW_frac"][ind] = list(help_df["MW_frac"])[0]
df_kcat.head(2)

#### (d) Creating a new DataFrame with one entry for every unique sequence-reaction pair:

##### (d)(i) Creating the DataFrame:

In [ ]:
df_kcat_new = pd.DataFrame(data = {"Reaction ID" : df_kcat["Reaction ID"],
                                  "Sequence ID" : df_kcat["Sequence ID"]})
df_kcat_new = df_kcat_new.loc[~pd.isnull(df_kcat_new["Reaction ID"])].loc[~pd.isnull(df_kcat_new["Sequence ID"])]
df_kcat_new.drop_duplicates(inplace = True)
df_kcat_new.reset_index(inplace = True, drop = True)


df_kcat_new["kcat_values"], df_kcat_new["Uniprot IDs"] = "", ""
df_kcat_new["from_BRENDA"], df_kcat_new["from_Sabio"], df_kcat_new["from_Uniprot"] = "", "", ""
df_kcat_new["checked"] = ""

for ind in df_kcat_new.index:
    RID, SID = df_kcat_new["Reaction ID"][ind], df_kcat_new["Sequence ID"][ind]
    help_df = df_kcat.loc[df_kcat["Reaction ID"] == RID].loc[df_kcat["Sequence ID"] == SID]

    df_kcat_new["kcat_values"][ind] = list(help_df["kcat"])
    df_kcat_new["Uniprot IDs"][ind] = list(help_df["Uniprot ID"])
    df_kcat_new["from_BRENDA"][ind] = list(help_df["from BRENDA"])
    df_kcat_new["from_Sabio"][ind] = list(help_df["from Sabio"])
    df_kcat_new["from_Uniprot"][ind] = list(help_df["from Uniprot"])
    df_kcat_new["checked"][ind] = list(help_df["checked"])
df_kcat_new

##### (d)(ii): Adding sequence, substrates, and products to all data points

In [ ]:
df_kcat_new["Sequence"], df_kcat_new["substrates"], df_kcat_new["products"], df_kcat_new["MW_frac"] = "", "", "", ""

for ind in df_kcat_new.index:
    RID, SID = df_kcat_new["Reaction ID"][ind], df_kcat_new["Sequence ID"][ind]
    help_df = df_reactions.loc[df_reactions["Reaction ID"] == RID]
    df_kcat_new["substrates"][ind], df_kcat_new["products"][ind] = list(help_df["substrate_InChI_set"])[0], list(help_df["product_InChI_set"])[0]
    df_kcat_new["MW_frac"][ind] = list(help_df["MW_frac"])[0]

    help_df = df_sequences.loc[df_sequences["Sequence ID"] == SID]
    df_kcat_new["Sequence"][ind] = list(help_df["Sequence"])[0]


##### (d)(iii) Calculating the maximal kcat value for every sequence and for every reaction:

In [ ]:
df_all_kcat = pd.read_pickle(join("..", "..", "data", "kcat_data", "kcat_data_merged.pkl"))
df_all_kcat.head()

df_kcat_new["max_kcat_for_UID"]  = ""
df_kcat_new["max_kcat_for_RID"]  = ""

for ind in df_kcat_new.index:
    max_kcat = - np.inf
    UIDs = list(set(df_kcat_new["Uniprot IDs"][ind]))
    for UID in UIDs:
        all_kcat = list(df_all_kcat["kcat"].loc[df_all_kcat["Uniprot ID"] == UID])
        all_kcat = [float(kcat) for kcat in all_kcat]
        max_kcat = max(max_kcat, max(all_kcat))
    df_kcat_new["max_kcat_for_UID"][ind] = max_kcat


for ind in df_kcat_new.index:
    RID = df_kcat_new["Reaction ID"][ind]

    help_df = df_kcat_new.loc[df_kcat_new["Reaction ID"] == RID]
    all_kcat = []
    for ind2 in help_df.index:
        all_kcat = all_kcat + list(help_df["kcat_values"][ind2])
    all_kcat = [float(kcat) for kcat in all_kcat]
    max_kcat = max(all_kcat)
    df_kcat_new["max_kcat_for_RID"][ind] = max_kcat
df_kcat_new.head()

,Reaction ID,Sequence ID,kcat_values,Uniprot IDs,from_BRENDA,from_Sabio,from_Uniprot,checked,Sequence,substrates,products,MW_frac,max_kcat_for_UID,max_kcat_for_RID
0,Reaction_0,Sequence_2804,"[2.8, 0.05, 0.11, 205.0, 2.3, 134.0, 360.0]","[P20932, P20932, P20932, P20932, P20932, P2093...","[0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0]","[False, False, False, False, False, False, False]",MSQNLFNVEDYRKLRQKRLPKMVYDYLEGGAEDEYGVKHNRDVFQQ...,{InChI=1S/C17H21N4O9P/c1-7-3-9-10(4-8(7)2)21(1...,{InChI=1S/C8H6O3/c9-7(8(10)11)6-4-2-1-3-5-6/h1...,1.0,360.0,360.0
1,Reaction_1,Sequence_2804,"[1.2, 3.4, 0.61, 0.07]","[P20932, P20932, P20932, P20932]","[0, 0, 0, 0]","[1, 1, 1, 1]","[0, 0, 0, 0]","[False, False, False, False]",MSQNLFNVEDYRKLRQKRLPKMVYDYLEGGAEDEYGVKHNRDVFQQ...,"{InChI=1S/O2/c1-2, InChI=1S/C17H23N4O9P/c1-7-3...",{InChI=1S/C17H21N4O9P/c1-7-3-9-10(4-8(7)2)21(1...,1.0,360.0,3.4
2,Reaction_2,Sequence_654,"[6.18, 14.5, 11.58, 13.12, 11.9, 13.98, 14.08,...","[P0ABQ4, P0ABQ4, P0ABQ4, P0ABQ4, P0ABQ4, P0ABQ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[False, False, False, False, False, False, Fal...",MISLIAALAVDRVIGMENAMPWNLPADLAWFKRNTLNKPVIMGRHT...,{InChI=1S/C21H28N7O17P3/c22-17-12-19(25-7-24-1...,{InChI=1S/C19H21N7O6/c20-19-25-15-14(17(30)26-...,1.0,29.0,29.0
3,Reaction_4,Sequence_3757,"[57.1, 19.6, 5.96, 13.6, 26.4, 14.0, 41.1, 11....","[P25704, P25704, P25704, P25704, P25704, P2570...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[False, False, False, False, False, False, Fal...",MAMQKIFAREILDSRGNPTVEVDLHTAKGRFRAAVPSGASTGIYEA...,"{InChI=1S/C3H7O7P/c4-1-2(3(5)6)10-11(7,8)9/h2,...","{InChI=1S/H2O/h1H2, InChI=1S/C3H5O6P/c1-2(3(4)...",1.0,57.1,84.0
4,Reaction_5,Sequence_1849,"[2.98, 0.87]","[P04746, P04746]","[0, 0]","[1, 1]","[0, 0]","[False, False]",MKFFLLLFTIGFCWAQYSPNTQQGRTSIVHLFEWRWVDIALECERY...,{InChI=1S/C18H32O16/c19-1-4-7(22)8(23)12(27)17...,{InChI=1S/C12H22O11/c13-1-3-5(15)6(16)9(19)12(...,1.0,2.98,2.98


##### (d)(iv) Calculating the maximal kcat value for every EC number in the dataset:

In [ ]:
df_kcat = df_kcat_new.copy()

#Using the txt file and the Uniprot mapping service to get an EC number for every enzyme:
df_EC = pd.read_csv(join("..", "..", "data", "enzyme_data", "Uniprot_results_EC.tab"), sep = "\t")
df_EC.head()

,Entry,EC number,Uniprot ID
0,P05091,1.2.1.3,P05091
1,Q5L1B7,1.7.1.13,Q5L1B7
2,A0A1C3HPT0,4.1.1.39,A0A1C3HPT0
3,D0RZL3,1.14.12.10,D0RZL3
4,D0RZL4,1.14.12.10,D0RZL4


In [ ]:
df_kcat.head()
df_kcat["ECs"] = ""
for ind in df_kcat.index:
    UID = df_kcat["Uniprot IDs"][ind][0]
    try:
        df_kcat["ECs"][ind] = list(df_EC["EC number"].loc[df_EC["Uniprot ID"] == UID])[0].split("; ")
    except:
        df_kcat["ECs"][ind] = []
df_kcat.head(2)

,Reaction ID,Sequence ID,kcat_values,Uniprot IDs,from_BRENDA,from_Sabio,from_Uniprot,checked,Sequence,substrates,products,MW_frac,max_kcat_for_UID,max_kcat_for_RID,ECs
0,Reaction_0,Sequence_2804,"[2.8, 0.05, 0.11, 205.0, 2.3, 134.0, 360.0]","[P20932, P20932, P20932, P20932, P20932, P2093...","[0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0]","[False, False, False, False, False, False, False]",MSQNLFNVEDYRKLRQKRLPKMVYDYLEGGAEDEYGVKHNRDVFQQ...,{InChI=1S/C17H21N4O9P/c1-7-3-9-10(4-8(7)2)21(1...,{InChI=1S/C8H6O3/c9-7(8(10)11)6-4-2-1-3-5-6/h1...,1.0,360.0,360.0,[1.1.99.31]
1,Reaction_1,Sequence_2804,"[1.2, 3.4, 0.61, 0.07]","[P20932, P20932, P20932, P20932]","[0, 0, 0, 0]","[1, 1, 1, 1]","[0, 0, 0, 0]","[False, False, False, False]",MSQNLFNVEDYRKLRQKRLPKMVYDYLEGGAEDEYGVKHNRDVFQQ...,"{InChI=1S/O2/c1-2, InChI=1S/C17H23N4O9P/c1-7-3...",{InChI=1S/C17H21N4O9P/c1-7-3-9-10(4-8(7)2)21(1...,1.0,360.0,3.4,[1.1.99.31]


In [ ]:
all_ECs = []
for ind in df_kcat.index:
    all_ECs = all_ECs + df_kcat["ECs"][ind]

all_ECs = list(set(all_ECs))

df_EC_kcat = pd.DataFrame({"EC" : all_ECs})
df_EC_kcat["max_kcat"] = np.nan

for ind in df_EC_kcat.index:
    try:
        kcat_max = get_max_for_EC_number(EC = df_EC_kcat["EC"][ind])
        df_EC_kcat["max_kcat"][ind] = kcat_max
        print(ind, kcat_max)
    except:
        pass

df_EC_kcat.to_pickle(join("..", "..", "data", "enzyme_data", "df_EC_max_kcat.pkl"))

0 nan
1 0.81
2 nan
3 2.0
4 3226.0
5 667.0
6 730.0
7 358.4
8 1.43
9 nan
10 3.5
11 43.5
12 1.12
13 221.0
14 122.0
15 295000.0
16 2135.0
17 32.0
18 45.0
19 3.72
20 2.69
21 4.1
22 9700.0
23 0.518
24 94.8
25 116.7
26 698.0
27 5.0
28 0.66
29 15.0
30 44.0
31 176.0
32 1600.0
33 258.3
34 5096.0
35 450.0
36 nan
37 nan
38 4.4
39 nan
40 2.82
41 nan
42 nan
43 1478.0
44 181.0
45 361.0
46 100.0
47 1230.0
48 55.0
49 2660.0
50 131.0
51 10.5
52 64.5
53 167.0
54 nan
55 22.0
56 4.83
57 3.1
58 1.2
59 nan
60 nan
61 778.5
62 0.87
63 2610.0
64 24.0
65 0.87
66 410.0
67 34.28
68 3330.0
69 1.73
70 nan
71 22000.0
72 246.0
73 4.1
74 9368.0
75 0.056
76 29.0
77 2644.0
78 nan
79 63.4
80 7.33
81 26.0
82 0.817
83 7.7
84 3.4
85 0.88
86 38.4
87 3850.0
88 118.0
89 nan
90 nan
91 nan
92 nan
93 75.8
94 nan
95 3.3
96 2201.64
97 8.82
98 5.9
99 0.142
100 75.0
101 0.888
102 nan
103 0.43
104 3350.0
105 124.5
106 0.8
107 nan
108 nan
109 9400.0
110 0.071
111 150.0
112 270.0
113 149.0
114 4.2
115 0.11
116 nan
117 330.0
118 432.0
119

880 60.8
881 nan
882 280.0
883 40000.0
884 1550.0
885 55.11
886 792.0
887 1340.0
888 0.913
889 1910.0
890 16.5
891 10.2
892 0.5
893 122.0
894 1.23
895 60.5
896 473.0
897 0.026
898 18.1
899 2170.0
900 90.0
901 81.0
902 72000.0
903 371.0
904 1660.0
905 5492.88
906 128.0
907 35.6
908 0.113
909 0.55
910 3657.0
911 2310.0
912 0.04
913 160.0
914 43.5
915 7300000.0
916 2871.0
917 275.0
918 54.1
919 1250.0
920 11.0
921 nan
922 2343.0
923 2085.0
924 212.0
925 136.3
926 88.0
927 1.0
928 6.08
929 171.0
930 480.0
931 16.0
932 96.0
933 16.7
934 2.5
935 47.0
936 642.9
937 2083.0
938 nan
939 nan
940 0.028
941 0.462
942 4.6
943 nan
944 1462.0
945 570000.0
946 22.0
947 1.6
948 5.9
949 11.1
950 2.0
951 208.3
952 145.0
953 10.2
954 0.31
955 125.2
956 11.4
957 0.036
958 8.1
959 160.0
960 63.0
961 9.6
962 4080.0
963 250.0
964 1612.0
965 0.49
966 nan
967 nan
968 39.1
969 313.0
970 487.0
971 nan
972 6.3
973 243.0
974 2860.0
975 1150.0
976 1012.0
977 2.7
978 410000.0
979 7000.0
980 140.5
981 2.0
982 0.11
983 

1673 190.0
1674 9.5
1675 10000.0
1676 341.0
1677 210.8
1678 nan
1679 1.0
1680 15.0
1681 16.7
1682 12.0
1683 12.2
1684 110.0
1685 69.7
1686 43.79
1687 41.67
1688 43.0
1689 6480.0
1690 70.0
1691 1.5
1692 5370.0
1693 57.0
1694 16.1
1695 12.13
1696 0.168
1697 7.18
1698 11.1
1699 26.0
1700 nan
1701 7.9
1702 5.09
1703 0.00139
1704 nan
1705 38.0
1706 10.5
1707 nan
1708 3.3
1709 0.435
1710 4.7
1711 131.0
1712 649.0
1713 0.019
1714 0.88
1715 21300.0
1716 0.9
1717 nan
1718 177.0
1719 330.0
1720 615.0
1721 2200.0
1722 0.0031
1723 42.26
1724 775.0
1725 1300.0
1726 49.3
1727 0.83
1728 1.27
1729 0.03
1730 1100.0
1731 33000000.0
1732 4833.0
1733 2.7
1734 32.7
1735 2.6
1736 nan
1737 76.8
1738 nan
1739 411.0
1740 0.0658
1741 nan
1742 97.5
1743 303.0
1744 430.0
1745 3.9
1746 910000.0
1747 3.67
1748 32.0
1749 nan
1750 0.053
1751 259.5
1752 0.86
1753 4400.0
1754 nan
1755 14400.0
1756 162.0
1757 0.15
1758 6.0
1759 0.9
1760 nan
1761 10.0
1762 1.54
1763 8.9
1764 18.3
1765 355200.0
1766 14.406
1767 450.0
1768

Mapping max EC kcat value to all data points:

In [ ]:
df_EC_kcat = pd.read_pickle(join("..", "..", "data", "enzyme_data", "df_EC_max_kcat.pkl"))
df_kcat["max_kcat_for_EC"] = np.nan

for ind in df_kcat.index:
    ECs = df_kcat["ECs"][ind]
    max_kcat = 0
    for EC in ECs:
        try:
            max_kcat = max(max_kcat, list(df_EC_kcat["max_kcat"].loc[df_EC_kcat["EC"] == EC])[0])
        except:
            pass
    if max_kcat != 0:
        df_kcat["max_kcat_for_EC"][ind] = max_kcat
df_kcat.to_pickle(join("..", "..", "data", "kcat_data", "merged_and_grouped_kcat_dataset.pkl"))

In [ ]:
df_sequences.to_pickle(join("..", "..", "data", "enzyme_data", "all_sequences_with_IDs.pkl"))
df_reactions.to_pickle(join("..", "..", "data", "reaction_data", "all_reactions_with_IDs.pkl"))

## 5. Calculating reaction fingerprints (structural and difference) for every reaction and a ESM-1b/ESM-1b_ts vector for every amino acid sequence:

#### (a) Executing jupyter notebook A2 to calculate the reaction fingerprints and enzyme representations. Then loading the results

In [6]:
df_kcat = pd.read_pickle(join("..", "..", "data", "kcat_data", "merged_and_grouped_kcat_dataset.pkl"))
df_kcat.head()

,Reaction ID,Sequence ID,kcat_values,Uniprot IDs,from_BRENDA,from_Sabio,from_Uniprot,checked,Sequence,substrates,products,MW_frac,max_kcat_for_UID,max_kcat_for_RID,ECs,max_kcat_for_EC
0,Reaction_0,Sequence_336,"[2.8, 0.05, 0.11, 205.0, 2.3, 134.0, 360.0]","[P20932, P20932, P20932, P20932, P20932, P2093...","[0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0]","[False, False, False, False, False, False, False]",MSQNLFNVEDYRKLRQKRLPKMVYDYLEGGAEDEYGVKHNRDVFQQ...,{InChI=1S/C17H21N4O9P/c1-7-3-9-10(4-8(7)2)21(1...,{InChI=1S/C17H23N4O9P/c1-7-3-9-10(4-8(7)2)21(1...,1.0,360.0,360.0,[1.1.99.31],550.0
1,Reaction_1,Sequence_336,"[1.2, 3.4, 0.61, 0.07]","[P20932, P20932, P20932, P20932]","[0, 0, 0, 0]","[1, 1, 1, 1]","[0, 0, 0, 0]","[False, False, False, False]",MSQNLFNVEDYRKLRQKRLPKMVYDYLEGGAEDEYGVKHNRDVFQQ...,"{InChI=1S/O2/c1-2, InChI=1S/C17H23N4O9P/c1-7-3...",{InChI=1S/C17H21N4O9P/c1-7-3-9-10(4-8(7)2)21(1...,1.0,360.0,3.4,[1.1.99.31],550.0
2,Reaction_2,Sequence_3907,"[6.18, 14.5, 11.58, 13.12, 11.9, 13.98, 14.08,...","[P0ABQ4, P0ABQ4, P0ABQ4, P0ABQ4, P0ABQ4, P0ABQ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[False, False, False, False, False, False, Fal...",MISLIAALAVDRVIGMENAMPWNLPADLAWFKRNTLNKPVIMGRHT...,{InChI=1S/C19H23N7O6/c20-19-25-15-14(17(30)26-...,{InChI=1S/C19H21N7O6/c20-19-25-15-14(17(30)26-...,1.0,29.0,29.0,[1.5.1.3],284.0
3,Reaction_4,Sequence_3802,"[57.1, 19.6, 5.96, 13.6, 26.4, 14.0, 41.1, 11....","[P25704, P25704, P25704, P25704, P25704, P2570...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[False, False, False, False, False, False, Fal...",MAMQKIFAREILDSRGNPTVEVDLHTAKGRFRAAVPSGASTGIYEA...,"{InChI=1S/C3H7O7P/c4-1-2(3(5)6)10-11(7,8)9/h2,...","{InChI=1S/H2O/h1H2, InChI=1S/C3H5O6P/c1-2(3(4)...",1.0,57.1,84.0,[4.2.1.11],230.0
4,Reaction_5,Sequence_2189,"[2.98, 0.87]","[P04746, P04746]","[0, 0]","[1, 1]","[0, 0]","[False, False]",MKFFLLLFTIGFCWAQYSPNTQQGRTSIVHLFEWRWVDIALECERY...,"{InChI=1S/H2O/h1H2, InChI=1S/C18H32O16/c19-1-4...",{InChI=1S/C6H12O6/c7-1-2-3(8)4(9)5(10)6(11)12-...,1.0,2.98,2.98,[3.2.1.1],40000.0


In [7]:
columns = df_kcat.columns

In [8]:
df_sequences = pd.read_pickle(join("..", "..", "data", "enzyme_data", "all_sequences_with_IDs_and_ESM1b_ts.pkl"))
df_reactions = pd.read_pickle(join("..", "..", "data", "reaction_data", "all_reactions_with_IDs_and_FPs.pkl"))

df_sequences = df_sequences[['Sequence', 'ESM1b', 'ESM1b_ts']]

In [9]:
len(df_reactions), len(df_sequences)

(4439, 3947)

In [ ]:
new_fet = pd.read_pickle(join("..", "..", "data", "kcat_data", "new_features.pkl"))
new_fet.head()

#### (b) Mapping ESM-1b vectors and reaction fingerprints to kcat dataset:

**Mapping Enzyme Sequence to Numeric feature representation**

In [ ]:
merged_sequences_df = pd.merge(df_kcat, df_sequences, on="Sequence")
merged_sequences_df

In [ ]:
merged_sequences_df.shape

In [ ]:
new_fet.iloc[:, 16:].columns

In [ ]:
merged_sequences_df = merged_sequences_df.join(new_fet.iloc[:, 16:])

In [ ]:
merged_sequences_df

**Finding fingerprints (DRFP, Structural, difference) vectors for each substrate and product of chemical reaction**

In [ ]:
mol_folder = join("..", "..", "data", "metabolite_data", "mol-files")

def get_reaction_site_smarts(metabolites):
    reaction_site = ""
    for met in metabolites:
        is_kegg_id = False

        if met[0] == "C":
            is_kegg_id = True

        if is_kegg_id:
            try:
                Smarts = Chem.MolToSmarts(Chem.MolFromMolFile(join(mol_folder, met + '.mol')))
            except OSError:
                return(np.nan)
        else:
            mol = Chem.inchi.MolFromInchi(met)
            if mol is not None:
                Smarts = Chem.MolToSmarts(mol)
            else:
                return(np.nan)
        reaction_site = reaction_site + "." + Smarts
    return(reaction_site[1:])


def get_reaction_site_smiles(metabolites):
    reaction_site = ""
    for met in metabolites:
        is_kegg_id = False

        if met[0] == "C":
            is_kegg_id = True

        if is_kegg_id:
            try:
                Smarts = Chem.MolToSmiles(Chem.MolFromMolFile(join(mol_folder, met + '.mol')))
            except OSError:
                return(np.nan)
        else:
            mol = Chem.inchi.MolFromInchi(met)
            if mol is not None:
                Smarts = Chem.MolToSmiles(mol)
            else:
                return(np.nan)
        reaction_site = reaction_site + "." + Smarts
    return(reaction_site[1:])


def convert_fp_to_array(difference_fp_dict):
    fp = np.zeros(2048)
    for key in difference_fp_dict.keys():
        fp[key] = difference_fp_dict[key]
    return(fp)


merged_sequences_df["structural_fp"], merged_sequences_df["difference_fp"], merged_sequences_df["DRFP"] = "", "", ""
merged_sequences_df["#substrates"], merged_sequences_df["#products"] = "", ""

for ind in merged_sequences_df.index:
    substrates = list(merged_sequences_df["substrates"][ind])
    products = list(merged_sequences_df["products"][ind])
    try:
        left_site = get_reaction_site_smarts(substrates)
        right_site = get_reaction_site_smarts(products)
        if not pd.isnull(left_site) and not pd.isnull(right_site):

            rxn_forward = AllChem.ReactionFromSmarts(left_site + ">>" + right_site)

            difference_fp = Chem.rdChemReactions.CreateDifferenceFingerprintForReaction(rxn_forward)
            difference_fp = convert_fp_to_array(difference_fp.GetNonzeroElements())
            structural_fp = Chem.rdChemReactions.CreateStructuralFingerprintForReaction(rxn_forward).ToBitString()

            left_site = get_reaction_site_smiles(substrates)
            right_site = get_reaction_site_smiles(products)
            drfp = DrfpEncoder.encode(left_site + ">>" + right_site)[0]

            merged_sequences_df["DRFP"][ind] = drfp
            merged_sequences_df["structural_fp"][ind] = structural_fp
            merged_sequences_df["difference_fp"][ind] = difference_fp
            merged_sequences_df["#substrates"][ind] = len(substrates)
            merged_sequences_df["#products"][ind] = len(products)

    except IndexError:
        pass


df_kcat = merged_sequences_df.copy()
n = len(df_kcat)
df_kcat["structural_fp"] = df_kcat["structural_fp"].astype(str)
df_kcat["ESM1b"] = df_kcat["ESM1b"].astype(str)
df_kcat["ESM1b_ts"] = df_kcat["ESM1b_ts"].astype(str)
#Remove values with missing reaction fingerprints or enzyme representation
df_kcat = df_kcat.loc[df_kcat["structural_fp"] != ""].loc[df_kcat["ESM1b"] != ""].loc[df_kcat["ESM1b_ts"] != ""]
print("Removing %s enzyme-reaction combinations because they either do not have a ESM1b vector or reaction fingerprint" % (n-len(df_kcat)))

In [ ]:
def convert_str_to_list(str_fp):
  '''
  converting string bit to list of bits
  '''
  fp_list = [int(bit) for bit in str_fp]
  return fp_list


df_kcat['structural_fp'] = df_kcat['structural_fp'].apply(convert_str_to_list)

In [ ]:
df_kcat.to_pickle(join("..", "..", "data", "kcat_data", "merged_and_grouped_kcat_dataset_with_FPs_and_ESM1bs_ts.pkl"))

## 6.Removing outliers and non-natural reactions:

In [ ]:
df_kcat = pd.read_pickle(join("..", "..", "data", "kcat_data", "merged_and_grouped_kcat_dataset_with_FPs_and_ESM1bs_ts.pkl"))

In [ ]:
np.array(df_kcat["kcat_values"][0]).astype(float)

#### (a) Calculating the geometric mean and log10-transforming it for all enzyme-reaction pairs:
To ignore $k_{cat}$ values that were obtained under non-optimal conditions, we exclude values lower than 1\% than the maximal $k_{cat}$ value for the same enzyme-reaction combination.

In [ ]:
df_kcat["geomean_kcat"] = np.nan
df_kcat["frac_of_max_UID"] = np.nan
df_kcat["frac_of_max_RID"] = np.nan
df_kcat["frac_of_max_EC"] = np.nan

for ind in df_kcat.index:
    all_kcat = np.array(df_kcat["kcat_values"][ind]).astype(float)
    max_kcat = max(all_kcat)
    all_kcat_top = [kcat for kcat in all_kcat  if kcat/max_kcat > 0.01]
    df_kcat["geomean_kcat"][ind] = np.mean(np.log10(all_kcat_top))

    df_kcat["frac_of_max_UID"][ind] =  np.max(np.array(df_kcat["kcat_values"][ind]).astype(float))/df_kcat["max_kcat_for_UID"][ind]
    df_kcat["frac_of_max_RID"][ind] =  np.max(np.array(df_kcat["kcat_values"][ind]).astype(float))/df_kcat["max_kcat_for_RID"][ind]
    df_kcat["frac_of_max_EC"][ind] = np.max(np.array(df_kcat["kcat_values"][ind]).astype(float))/df_kcat["max_kcat_for_EC"][ind]
df_kcat = df_kcat.loc[~pd.isnull(df_kcat["geomean_kcat"])]

len(df_kcat)

#### (b) We are only interested in kcat values that were measured for the natural reaction of an enzyme:
To achieve this we exclude kcat values for an enzyme if another measurement exists for the same enzyme but for different reaction with a kcat value that is more than ten times higher. Furthermore, to exlcude data points measured under non-optimal conditions and for non-natural reactions, we exclude kcat values if we could find a kcat value for the same reaction or same EC number that is more than 100 times higher.

In [ ]:
n = len(df_kcat)

df = df_kcat.loc[(df_kcat["frac_of_max_UID"] < 0.1) | (df_kcat["frac_of_max_UID"] == 0.1)]
df_kcat = df_kcat.loc[df_kcat["frac_of_max_UID"] > 0.1]
df = pd.concat([df, df_kcat[(df_kcat["frac_of_max_RID"] < 0.01) | (df_kcat["frac_of_max_RID"] == 0.01)]], ignore_index=True)
df_kcat = df_kcat.loc[df_kcat["frac_of_max_RID"] > 0.01]

df_kcat["frac_of_max_EC"].loc[pd.isnull(df_kcat["frac_of_max_EC"])] = 1
df = pd.concat([df, df_kcat.loc[(df_kcat["frac_of_max_EC"] > 10) | (df_kcat["frac_of_max_EC"] == 10)]], ignore_index=True)
df_kcat = df_kcat.loc[df_kcat["frac_of_max_EC"] < 10]
df = pd.concat([df, df_kcat.loc[(df_kcat["frac_of_max_EC"] < 0.01) | (df_kcat["frac_of_max_EC"] == 0.01)]], ignore_index=True)
df_kcat = df_kcat.loc[df_kcat["frac_of_max_EC"] > 0.01]
df.shape

In [ ]:
print("We remove %s data points, because we suspect that these kcat values were not measure for the natural reaction " \
    "of an enzyme or under non-optimal conditions." % (n-len(df_kcat)))

#### (c) Removing data points with reaction queations with uneven fraction of molecular weights

In [ ]:
n = len(df_kcat)

df_kcat = df_kcat.loc[df_kcat["MW_frac"] < 3]
df_kcat = df_kcat.loc[df_kcat["MW_frac"] > 1/3]

print("We remove %s data points because the sum of molecular weights of substrates does not match the sum of molecular" \
      "weights of the products." % (n-len(df_kcat)))

#### (d) Removing data points with outlying kcat values:

In [ ]:
n = len(df_kcat)

df_kcat = df_kcat.loc[~(df_kcat["geomean_kcat"]>5)]
df_kcat = df_kcat.loc[~(df_kcat["geomean_kcat"]<-2.5)]

print("We remove %s data point because their kcat values are outliers." % (n-len(df_kcat)))

## 7. Normalizing the Input and Ouput Features

**Normalizing ESM1b feat, with mean = 0 and std = 1**

Input Features Normalization

In [ ]:
esm1b = np.array(df_kcat['ESM1b'].tolist())
esm1b_ts = np.array(df_kcat['ESM1b_ts'].tolist())

esm1b_mean, esm1b_std = esm1b.mean(), esm1b.std()
esm1b_ts_mean, esm1b_ts_std = esm1b_ts.mean(), esm1b_ts.std()


def normalize_esm1b(feat_list):
    norm_feat_list = (feat_list - esm1b_mean)/esm1b_std
    return norm_feat_list


def normalize_esm1b_ts(feat_list):
    norm_feat_list = (feat_list - esm1b_ts_mean)/esm1b_ts_std
    return norm_feat_list


df_kcat['ESM1b_norm'] = df_kcat['ESM1b'].apply(normalize_esm1b)
df_kcat['ESM1b_ts_norm'] = df_kcat['ESM1b_ts'].apply(normalize_esm1b_ts)


Output Feature Normalization

In [ ]:

kcat_mean = df_kcat['geomean_kcat'].mean()
kcat_std = df_kcat['geomean_kcat'].std()


def normalize_kcat(kcat):
    kcat_norm = (kcat - kcat_mean)/kcat_std
    return kcat_norm

df_kcat['log10_kcat_norm'] = df_kcat['geomean_kcat'].apply(normalize_kcat)


In [ ]:
print("Size of final kcat dataset: %s" % len(df_kcat))
df_kcat.to_pickle(join("..", "..", "data", "kcat_data", "final_kcat_dataset_new.pkl"))

## 8. Splitting the dataset into training and test set:

#### (a) Splitting the dataset in such a way that the same enzyme does not occur in the training and the test set:

In [ ]:
df_kcat = pd.read_pickle(join("..", "..", "data", "kcat_data", "final_kcat_dataset_new.pkl"))
df_kcat.head()

Shuffling DataFrame:

In [ ]:
df = df_kcat.copy()
df = df.sample(frac = 1, random_state = 123)
df.reset_index(drop= True, inplace = True)

Splitting dataset

In [ ]:
train_df, test_df = split_dataframe_enzyme(frac = 5, df = df.copy())
print("Test set size: %s" % len(test_df))
print("Training set size: %s" % len(train_df))
print("Size of test set in percent: %s" % np.round(100*len(test_df)/ (len(test_df) + len(train_df))))


train_df.reset_index(inplace = True, drop = True)
test_df.reset_index(inplace = True, drop = True)

train_df.to_pickle(join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_fet.pkl"))
test_df.to_pickle(join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_fet.pkl"))
train_df.to_csv(join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_fet.csv"), index=False)
test_df.to_csv(join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_fet.csv"), index=False)

#### (b) Splitting the training set into 5 folds for 5-fold cross-validations (CVs):
In order to achieve a model that generalizes well during CV, we created the 5 folds in such a way that neither the same enzyme nor the same reaction occurs in two different subsets.

In [ ]:
train_df = pd.read_pickle(join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_fet.pkl"))
test_df = pd.read_pickle(join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_fet.pkl"))

In [ ]:
data_train2 = train_df.copy()
data_train2["index"] = list(data_train2.index)

data_train2, df_fold = split_dataframe_enzyme(df = data_train2, frac=5)
indices_fold1 = list(df_fold["index"])
print(len(data_train2), len(indices_fold1))#

data_train2, df_fold = split_dataframe_enzyme(df = data_train2, frac=4)
indices_fold2 = list(df_fold["index"])
print(len(data_train2), len(indices_fold2))

data_train2, df_fold = split_dataframe_enzyme(df = data_train2, frac=3)
indices_fold3 = list(df_fold["index"])
print(len(data_train2), len(indices_fold3))

data_train2, df_fold = split_dataframe_enzyme(df = data_train2, frac=2)
indices_fold4 = list(df_fold["index"])
indices_fold5 = list(data_train2["index"])
print(len(data_train2), len(indices_fold4))


fold_indices = [indices_fold1, indices_fold2, indices_fold3, indices_fold4, indices_fold5]

train_indices = [[], [], [], [], []]
test_indices = [[], [], [], [], []]

for i in range(5):
    for j in range(5):
        if i != j:
            train_indices[i] = train_indices[i] + fold_indices[j]

    test_indices[i] = fold_indices[i]


np.save(
    join("..", "..", "data", "kcat_data", "splits", "CV_train_indices_new"), train_indices)
np.save(
    join("..", "..", "data", "kcat_data", "splits", "CV_test_indices_new"), test_indices)